# Initialize Spark Session

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from os.path import abspath

# SparkSession
URL_SPARK = "spark://spark-master:7077"
warehouse_location = abspath('spark-warehouse')

spark = (
    SparkSession.builder
    .appName("spark-ingest-image")
    .config("executor.memory", "8g")
    .config("spark.sql.warehouse.dir", warehouse_location)
    .config("spark.jars", "jars/hadoop-aws-3.3.2.jar,jars/aws-java-sdk-bundle-1.12.172.jar,jars/hadoop-common-3.3.2.jar,jars/commons-pool2-2.11.1.jar,jars/spark-avro_2.12-3.3.2.jar")
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    .master(URL_SPARK)
    .getOrCreate()
)


/usr/local/lib/python3.9/dist-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found


24/01/04 17:17:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
import os
os.environ["AWS_ACCESS_KEY_ID"] = "minioadmin"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minioadmin"
# os.environ["MLFLOW_S3_ENDPOINT_URL"] = f"http://sparkjupyter-minio-1:9000"

spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://sparkjupyter-minio-1:9000")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.access.key", "minioadmin")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "minioadmin" )
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled","false")

In [3]:
path_to_MinIO = "s3a://test-data-eric/test/*"

In [4]:
from pyspark.sql.functions import *

df = spark.read.format("image").load(path_to_MinIO)

24/01/04 17:17:52 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [5]:
df.select("image.origin", "image.width", "image.height").show(truncate=False)

+-----------------------------------+-----+------+
|origin                             |width|height|
+-----------------------------------+-----+------+
|s3a://test-data-eric/test/test1.jpg|1280 |720   |
|s3a://test-data-eric/test/test5.jpg|1280 |720   |
|s3a://test-data-eric/test/test3.jpg|1280 |720   |
|s3a://test-data-eric/test/test4.jpg|1280 |720   |
|s3a://test-data-eric/test/test2.jpg|1280 |720   |
+-----------------------------------+-----+------+

